In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
import gif2numpy
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Dense, Input, InputLayer, Flatten
from tensorflow.keras.models import Sequential, Model
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
from sklearn.utils import shuffle
%matplotlib inline
import splitfolders

2023-03-29 22:31:38.363092: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [120]:
# split into training and testing sets 
splitfolders.ratio("./pictures", output="profile_images_split", ratio=(.8, .1, .1), group_prefix=None, move=False) # default values

Copying files: 19746 files [00:18, 1072.81 files/s]


In [129]:
# split into training and testing sets 
splitfolders.ratio("./pictures2", output="images_split", ratio=(.8, .1, .1), group_prefix=None, move=False) # default values



Copying files: 10226 files [00:17, 574.02 files/s]


In [3]:
IMG_WIDTH = 200
IMG_HEIGHT = 200

def create_dataset(img_folder, bot=True):
    img_data_array = []
    class_name = []
    
    for dir1 in os.listdir(img_folder):
        image_path = os.path.join(img_folder, dir1)
        image = cv2.imread(image_path, cv2.COLOR_BGR2RGB)
        try:
            image = cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH), interpolation = cv2.INTER_AREA)
        except:
            image = gif2numpy.convert(image_path)[0][0]
            image = cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH), interpolation = cv2.INTER_AREA)
        image = np.array(image)
        image = image.astype('float32')
        image /= 255 
        img_data_array.append(image)
        if bot:
            class_name.append([1])
        else: 
            class_name.append([0])
    return img_data_array, class_name

# extract the image array and class name
bot_train, class_name_bot = create_dataset('./images_split/train/1_bot')
human_train, class_name_human = create_dataset('/Users/jess/Desktop/images_split/train/0_human', False)

bot_test, class_name_bot_test = create_dataset('/Users/jess/Desktop/images_split/test/1_bot')
human_test, class_name_human_test = create_dataset('/Users/jess/Desktop/images_split/test/0_human', False)

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


In [4]:
print(len(bot_train))
print(len(class_name_bot))
print(len(human_train))
print(len(class_name_human))

3300
3300
4880
4880


In [5]:
bot_train = np.array(bot_train, dtype=np.float32)
human_train = np.array(human_train, dtype=np.float32)
train_images = np.concatenate((bot_train, human_train), dtype=np.float32)
train_labels = np.array(class_name_bot+class_name_human, np.float32)

bot_test = np.array(bot_test, dtype=np.float32)
human_test = np.array(human_test, dtype=np.float32)
test_images = np.concatenate((bot_test, human_test), dtype=np.float32)
test_labels = np.array(class_name_bot_test+class_name_human_test, np.float32)

In [6]:
train_images, train_labels = shuffle(train_images, train_labels)
test_images, test_labels = shuffle(test_images, test_labels)

In [7]:
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

(15796, 200, 200, 3)
(15796, 1)
(3950, 200, 200, 3)
(3950, 1)
